In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import PIL
import cv2

from tensorflow import keras
from tensorflow.keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.optimizers import *
from tensorflow.keras import *
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D,BatchNormalization,Input, Lambda
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
IMAGE_SIZE = [224, 224]

In [3]:
train_path = "C:\\Users\\Nethmi Mohotti\\Desktop\\CobraShot\\Data Science\\dataset\\train"
test_path = "C:\\Users\\Nethmi Mohotti\\Desktop\\CobraShot\\Data Science\\dataset\\testing"

In [4]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [5]:
for layer in vgg.layers:
  layer.trainable = False

In [6]:
from glob import glob
folders = glob("C:\\Users\\Nethmi Mohotti\\Desktop\\CobraShot\\Data Science\\dataset\\train\\*")
print(len(folders))

7


In [7]:
x = Flatten()(vgg.output)

In [8]:
prediction = Dense(len(folders), activation='softmax')(x)

In [9]:
model = Model(inputs=vgg.input, outputs=prediction)

In [10]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [11]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [12]:
train_dataset = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_dataset = ImageDataGenerator(rescale = 1./255)

In [13]:
training_set = train_dataset.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

testing_set = test_dataset.flow_from_directory(test_path, 
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 2473 images belonging to 7 classes.
Found 83 images belonging to 7 classes.


In [14]:
training_set.class_indices

{'Black rat snake': 0,
 'Common garter snake': 1,
 'Common watersnake': 2,
 "Russell's Viper": 3,
 'Saw-scaled Viper': 4,
 'Spectacled Cobra': 5,
 'Western diamondback rattlesnake': 6}

In [15]:
training_set.classes

array([0, 0, 0, ..., 6, 6, 6])

In [16]:
r = model.fit_generator(
  training_set,
  validation_data=testing_set,
  epochs=30,
  steps_per_epoch=len(training_set),
  validation_steps=len(testing_set)
)

C:\Users\NETHMI~1\AppData\Local\Temp/ipykernel_19696/2614426977.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/30
78/78 [==============================] - 354s 5s/step - loss: 1.5949 - accuracy: 0.4683 - val_loss: 0.9174 - val_accuracy: 0.6627
Epoch 2/30
78/78 [==============================] - 352s 5s/step - loss: 0.9214 - accuracy: 0.6696 - val_loss: 0.9642 - val_accuracy: 0.6145
Epoch 3/30
78/78 [==============================] - 351s 4s/step - loss: 0.8346 - accuracy: 0.7020 - val_loss: 1.1427 - val_accuracy: 0.6024
Epoch 4/30
78/78 [==============================] - 352s 5s/step - loss: 0.7396 - accuracy: 0.7339 - val_loss: 1.0572 - val_accuracy: 0.6386
Epoch 5/30
78/78 [==============================] - 357s 5s/step - loss: 0.6318 - accuracy: 0.7715 - val_loss: 0.7642 - val_accuracy: 0.6747
Epoch 6/30
78/78 [==============================] - 348s 4s/step - loss: 0.5338 - accuracy: 0.8156 - val_loss: 0.8407 - val_accuracy: 0.6747
Epoch 7/30
78/78 [==============================] - 347s 4s/step - loss: 0.5191 - accuracy: 0.8221 - val_loss: 1.0125 - val_accuracy: 0.6386
Epoch 8/30
78

In [17]:
from tensorflow.keras.models import load_model
model.save('cobrashot_model.h5')

In [18]:
from keras.models import load_model
cobrashot_model = load_model('cobrashot_model.h5')

In [19]:
import matplotlib.pyplot as plt
%matplotlib inline
#list all data in training
print(cobrashot_model.history.keys())
# summarize training for accuracy
plt.plot(cobrashot_model.history['accuracy'])
plt.plot(cobrashot_model.history['val_accuracy'])
plt.title('Accuracy graph')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(['train','test'], loc='upper left')
plt.show()

# summarize training for loss
plt.plot(cobrashot_model.history['loss'])
plt.plot(cobrashot_model.history['val_loss'])
plt.title('Loss graph')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train','test'], loc='upper left')
plt.show()

AttributeError: 'NoneType' object has no attribute 'keys'